In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from my_functions import *
from three_filter import three_filter_method, NB_3fm


In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_30deg_uniform_g2.npy')
pm_data_noerr = np.load('npy/pm_LAE_30deg_uniform_g2.npy')
f_name = 'Source_cats/Source_cat_LAE_30deg_uniform_g2.npy'
mock = np.load(f_name, allow_pickle=True).item()

## Somehow, some bright mocks are sneaking in. So, get rid of them.
mask_bright = np.where(pm_data_noerr[-3] <= 1e-17)[0]
pm_data_noerr = pm_data_noerr[:, mask_bright]
pm_data = pm_data[:, mask_bright]
for key in mock.keys():
    if key == 'w_Arr': continue
    mock[key] = mock[key][mask_bright]

err_fit_params = np.load('npy/err_fit_params_minijpas.npy')
m = err_fit_params[:, 0].reshape(-1, 1)
b = err_fit_params[:, 1].reshape(-1, 1)
pm_err = pm_data_noerr * 10 ** (b + m * np.log10(np.abs(pm_data_noerr)))

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
## 3 Filter method
ew0min = 0
ll = []
for nb_c in range(3, 18):
    NB = pm_data[nb_c]
    BB_LC = pm_data[-3]
    BB_LU = pm_data[-2]
    NB_err = pm_err[nb_c]
    BB_LC_err = pm_err[-3]
    BB_LU_err = pm_err[-2]
    t_NB = np.array(tcurves['t'][nb_c])
    t_BB_LC = np.array(tcurves['t'][-3])
    t_BB_LU = np.array(tcurves['t'][-2])
    w_NB = np.array(tcurves['w'][nb_c])
    w_BB_LC = np.array(tcurves['w'][-3])
    w_BB_LU = np.array(tcurves['w'][-2])
    w_EL = w_central[nb_c]

    F_EL, A, B, A_err, B_err = three_filter_method(
        NB, BB_LC, BB_LU, NB_err, BB_LC_err, BB_LU_err,
        t_NB, w_NB, t_BB_LC, t_BB_LU, w_BB_LC, w_BB_LU, w_EL
    )

    f_cont = A * w_EL + B
    f_cont_var = w_EL**2 * A_err**2 + B_err**2
    EW0 = F_EL / f_cont * 1215.67/w_EL

    line = (
        (pm_data[nb_c] - f_cont > 3*(pm_err[nb_c]**2 + f_cont_var) ** 0.5)
        & (EW0 > ew0min)
    )
    print(
        '{0} Selected: {1:0.2f}%'
        .format(filters_tags[nb_c], len(np.where(line)[0]) / len(line) * 100)
    )
    ll.append(len(np.where(line)[0]) / len(line) * 100)
plt.plot(ll, marker='.', markersize=15, linestyle='')

In [ ]:
np.array(nb_fwhm(tcurves, -3, False)) / 1215.67 - 1

In [ ]:
plt.plot(np.average(pm_err[3:18], axis=1), marker='.', linestyle='', markersize=15)